In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
stashapp_galleries = stash_client.find_galleries_by_studio(6)

In [ ]:
xart_scenes = pl.read_parquet("H:\\Parquet Data\\X-Art\\xart_scenes.parquet")
xart_galleries = xart_scenes.filter(pl.col("type") == "gallery")

In [ ]:
joined_galleries = xart_galleries.join(stashapp_galleries, left_on="title", right_on="stashapp_title", how="left", coalesce=False)
joined_galleries.select(["title", "date", "performers", "description", "image_count", "stashapp_title", "stashapp_id", "stashapp_image_count"])

In [ ]:
existing_joined_galleries = joined_galleries.filter(pl.col("stashapp_id").is_not_null())
existing_joined_galleries = existing_joined_galleries.select(
    pl.when((pl.col("image_count") == pl.col("stashapp_image_count")) | (pl.col("image_count") - pl.col("stashapp_image_count") <= 1))
        .then(pl.lit(True))
        .otherwise(pl.lit(False))
        .alias("image_count_match"),
    pl.col("image_count"),
    pl.col("stashapp_image_count"),
    pl.col("title"),
    pl.col("date"),
    pl.col("performers"),
    pl.col("description"),
    pl.col("stashapp_id"),
    pl.col("stashapp_title"),    
)
existing_joined_galleries

In [ ]:
missing_joined_galleries = joined_galleries.filter(pl.col("stashapp_id").is_null())
missing_joined_galleries = missing_joined_galleries.unique(["title", "date", "performers", "description"])
missing_joined_galleries